
우리는 앞서 **선언 지점 변성(Declaration-site variance)** 에 대해 살펴봤습니다.
이번에는 **사용 지점 변성(Use-site variance)** 을 보겠습니다.

우선 `Car` 클래스를 정의하고, 상속할 수 있도록 `open` 키워드를 붙입니다:

```kotlin
open class Car

class Toyota : Car()
class Ford : Car()
```

이제 "리스트 복사 함수"를 작성해 보겠습니다. 아주 단순한 함수인데요:

```kotlin
fun copyCars(source: MutableList<Car>, destination: MutableList<Car>) {
    for (car in source) {
        destination.add(car)
    }
}
```

이제 `main`에서 호출해보겠습니다:

```kotlin
val cars1 = mutableListOf(Car(), Car())
val cars2 = mutableListOf<Car>()

copyCars(cars1, cars2) // OK
```

문제 없음. 잘 동작합니다.

---

하지만 이제 `Ford`만 담긴 리스트를 복사하려고 하면 문제가 생깁니다:

```kotlin
val fords1 = mutableListOf(Ford(), Ford())
val fords2 = mutableListOf<Ford>()

copyCars(fords1, fords2) // 에러 발생 ❌
```

왜냐하면 **제네릭 타입은 기본적으로 불공변(invariant)** 이기 때문입니다.
즉 `MutableList<Ford>`는 `MutableList<Car>`가 아닙니다. (`Ford`는 `Car`의 하위 타입이지만 제네릭에서는 그대로 적용되지 않음)

우리는 `Car`의 하위타입마다 `copyCars`를 따로 만들고 싶진 않습니다. 그래서 **제네릭**을 도입합니다:

```kotlin
fun <T> copyCars(source: MutableList<T>, destination: MutableList<T>) {
    for (car in source) {
        destination.add(car)
    }
}
```

이제 `Ford`와 `Car`에 대해서는 잘 동작하지만, 여전히 이런 건 불가능합니다:

```kotlin
copyCars(fords1, cars2) // 에러 ❌
```

`T`는 양쪽이 동일해야 하기 때문에 (`MutableList<Ford>` vs `MutableList<Car>`) 타입이 안 맞습니다.
하지만 사실 이런 건 되고 있어야 합니다:

```kotlin
val cars3: MutableList<Car> = mutableListOf(Ford(), Ford()) // OK
```

즉, `Ford` 리스트를 `Car` 리스트로 복사하는 건 합리적이지만 제네릭 제약 때문에 막힌 상황입니다.

---

여기서 **사용 지점 변성(Use-site variance)** 을 적용할 수 있습니다.

생각해보면:

* **원본 리스트(source)** 는 읽기만 합니다 → **공변(covariant)** 이 되어야 함 (`out`)
* **목적지 리스트(destination)** 는 쓰기만 합니다 → **반공변(contravariant)** 이 될 수 있음 (`in`)

그래서 함수 시그니처를 이렇게 바꿀 수 있습니다:

```kotlin
fun <T> copyCars(source: MutableList<out T>, destination: MutableList<T>) {
    for (car in source) {
        destination.add(car)
    }
}
```

이제 `copyCars(fords1, cars2)` 가 잘 작동합니다 ✅

---

정리하면:

* **공변(out)** → 읽기 전용, `T` 또는 하위 타입 허용
* **반공변(in)** → 쓰기 전용, `T` 또는 상위 타입 허용
* 함수 안에서 읽기/쓰기 제약이 딱 맞을 때 사용 지점 변성을 쓸 수 있음
* 이는 **Java의 와일드카드(extends / super)** 와 동일한 개념임

  * `? extends Car` ↔ Kotlin `out Car`
  * `? super Car` ↔ Kotlin `in Car`

Java에서는 클래스나 인터페이스 선언 시 변성을 지정할 수 없고, 매번 메소드에서 `? extends` 또는 `? super` 를 붙여야 함.
반면 Kotlin은 **선언 지점 변성**(클래스 정의할 때 `out` / `in` 지정)과 **사용 지점 변성**(함수 호출 시 `out` / `in` 지정)을 모두 지원함.

---

## 📌 핵심 정리

1. **문제 상황**

   * 제네릭은 기본적으로 불공변(invariant) → `MutableList<Ford>`는 `MutableList<Car>` 아님.
   * 그래서 `copyCars(fords, cars)` 같은 호출이 안 됨.

2. **해결책**

   * 원본 리스트는 "읽기 전용" → `out` 키워드 사용 (공변).
   * 목적지 리스트는 "쓰기 전용" → 필요하다면 `in` 키워드 사용 (반공변).

3. **Kotlin vs Java**

   * Kotlin: `out`, `in` → 선언 지점 + 사용 지점 모두 지원.
   * Java: `? extends`, `? super` → 오직 사용 지점에서만 지정 가능.

4. **정리 문장**
   👉 "사용 지점 변성(use-site variance)"은 함수 호출 시 제네릭 타입의 공변성/반공변성을 한정하는 방법이다.
   👉 이걸 쓰면 `copyCars(fords, cars)`처럼 하위타입을 상위타입에 맞게 자연스럽게 전달할 수 있다.

---

